In [1]:
import pandas as pd
import string

In [2]:
df = pd.read_csv('data/kc_cleaned_data.csv')
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year

# Import outside data

In [3]:
df_zipcode = pd.read_csv('data/zipcode.csv')
print(df_zipcode.shape)

df_zipcode.drop(columns=['Region Type', 'Type', 'Month Over Month',
       'Quarter Over Quarter', 'Year Over Year', '5 Year Annualized',
       '10 Year Annualized'], inplace=True)
df_zipcode.head()

(85, 9)


,Region Name,Current
0,King County,"$667,300"
1,98001,"$432,800"
2,98002,"$358,500"
3,98003,"$392,800"
4,98004,"$1,613,200"


In [4]:
df_loc=df.copy()

### Clean price column

In [5]:
def strip_col():
    replace_col = []
    index = 0
    while index < len(df_zipcode['Current']):
        new = (df_zipcode['Current'][index]).translate(str.maketrans('', '', string.punctuation))
        replace_col.append(new)
        index += 1
    return replace_col

In [6]:
df_zipcode['Current'] = strip_col()
df_zipcode['Current'] = pd.to_numeric(df_zipcode['Current'])

df_zipcode.drop(labels=0, axis=0, inplace=True)
df_zipcode['Region Name'] = pd.to_numeric(df_zipcode['Region Name'])
df_zipcode.head()

,Region Name,Current
1,98001,432800
2,98002,358500
3,98003,392800
4,98004,1613200
5,98005,1118500


# Merge

In [7]:
def add_zipdf_to_locdf():
    area_value = []
    index = 0
    while index < len(df_loc['zipcode']):
        loc_val = df_loc['zipcode'][index]
        this_current = (df_zipcode.loc[df_zipcode['Region Name']==loc_val])
        area_value.append((pd.Series(this_current['Current']).values)[0])
        index+=1
    return area_value

In [8]:
df_loc['Current'] = add_zipdf_to_locdf()
df_loc.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,zipcode,lat,long,sqft_living15,sqft_lot15,has_basement,is_cust_build,years_since_renovated,year,Current
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0.0,0.0,...,98178,47.5112,-122.257,1340,5650,0,0,0.0,2014,505100
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,98125,47.7210,-122.319,1690,7639,1,0,29.0,2014,685200
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,98028,47.7379,-122.233,2720,8062,0,0,0.0,2015,716200
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,98136,47.5208,-122.393,1360,5000,1,0,0.0,2014,769400
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,98074,47.6168,-122.045,1800,7503,0,0,0.0,2015,966900


In [9]:
df_loc.to_csv('data/kc_data_plus_imported.csv', index=False)

In [10]:
df_loc.drop(columns=['bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated',
       'sqft_living15', 'sqft_lot15', 'has_basement',
       'is_cust_build', 'years_since_renovated'], inplace=True)

In [11]:
df_loc.to_csv('data/location_price_info.csv', index=False)